In this notebook we merge the text dataset with the features previously computed in step E. 


In [1]:
# Goal entry format:
# [
#     unique_id (cik, gvkey, cusip, date),
#     risk_factors_text,
#     management_text,
#     std_dev,
#     realized_drawdown
# ]

# Columns in text data:
# ,index,date,cik,file_type,rf,mgmt,gvkey,cusip,year

# Columns in numerical data: 
# id,date,ret_eom,gvkey,excntry,stock_ret,prc,hist_vol,"realized maximum drawdown (1yr, forward)"
# where ret_eom is actually the date (last day of its month)

# So we need to merge on (gvkey, ret_eom/date)
# - num.ret_eom looks like: 2025-02-28 and is always the last day of the month
# - text.date   looks like: 20140102   and is always the exact filing date
# IDEA 1:
# -------
# We format text.date to YYYY-MM-DD, then use a new column year_month = YYYY-MM
# We will merge on (gvkey, year_month))

In [2]:
import pandas as pd
from datetime import datetime
from pathlib import Path

In [ ]:
NUM_DATA_PATH = Path("../data/features_with_labels.csv")
TEXT_DATA_DIR_PATH = Path("../data/text_data/medium (1_000 words)/")
SAVE_MERGED_PATH = Path("../data/ready/data.csv")
SAVE_MERGED_PATH.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
num_df = pd.read_csv(Path(NUM_DATA_PATH))
num_df["temp_date"] = pd.to_datetime(num_df["ret_eom"])
num_df["year_month"] = num_df["temp_date"].dt.to_period("M").astype(str)

data_df = pd.DataFrame(columns=["gvkey", "year_month", "risk_factors_text", "management_text", "std_dev", "realized_drawdown"])

for year in range(2005, 2026):
    text_file_path = TEXT_DATA_DIR_PATH / f"text_us_{year}_preprocessed.csv"
    if not text_file_path.exists():
        print(f"Warning: {text_file_path} does not exist. Skipping.")
        continue
    
    text_df = pd.read_csv(text_file_path)
    
    text_df["date"] = pd.to_datetime(text_df["date"], format="%Y%m%d")
    text_df["year_month"] = text_df["date"].dt.to_period("M").astype(str)
    
    merged_df = pd.merge(
        text_df,
        num_df,
        left_on=["gvkey", "year_month"],
        right_on=["gvkey", "year_month"],
        how="inner"
    )
    
    merged_df = merged_df[["gvkey", "year_month", "rf", "mgmt", "hist_vol", "realized maximum drawdown (1yr, forward)"]]
    merged_df.rename(columns={
        "hist_vol": "std_dev",
        "rf": "risk_factors_text",
        "mgmt": "management_text",
        "realized maximum drawdown (1yr, forward)": "realized_drawdown"
    }, inplace=True)
    
    data_df = pd.concat([data_df, merged_df], ignore_index=True)



C:\Users\willi\AppData\Local\Temp\ipykernel_17276\2482593731.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_df = pd.concat([data_df, merged_df], ignore_index=True)


In [5]:
data_df.describe()

,gvkey,std_dev,realized_drawdown
count,284484.000000,267494.000000,272308.000000
mean,65678.191955,0.467331,-0.302119
std,66277.977698,0.333589,0.215477
min,1004.000000,0.000000,-0.999938
25%,13354.000000,0.253287,-0.424118
50%,30614.000000,0.377278,-0.244364
75%,125794.000000,0.575183,-0.132911
max,349972.000000,3.681053,0.000000


In [6]:
data_df.dropna(inplace=True)
data_df.reset_index(drop=True, inplace=True)
data_df.describe()

,gvkey,std_dev,realized_drawdown
count,227634.000000,227634.000000,227634.000000
mean,67970.782265,0.470478,-0.306729
std,67317.178111,0.331696,0.216889
min,1004.000000,0.000000,-0.999938
25%,14253.000000,0.256875,-0.431035
50%,31381.000000,0.381751,-0.249621
75%,133944.000000,0.579744,-0.135828
max,349972.000000,3.681053,0.000000


In [7]:
data_df.to_csv(SAVE_MERGED_PATH, index=False)